In [ ]:
conda install -c conda-forge umap-learn

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
#from umap import UMAP 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [10]:
import nltk

In [11]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [12]:
from nltk.corpus import stopwords
sw = stopwords.words('english')

In [13]:
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

ModuleNotFoundError: No module named 'gensim'

In [ ]:
bible_words = [
    "thou", "thee", "thy", "thine", "ye", "you", "your", "yours", "shall",
    "art", "hath", "hast", "wilt", "shalt", "shouldst", "couldst", "canst",
    "dost", "doth", "didst", "were", "wast", "wert", "be", "been", "am", 
    "is", "are", "was", "unto", "of", "in", "for", "with", "on", "at", "by",
    "through", "to", "from", "up", "down", "out", "as", "so", "but", "and",
    "also", "therefore", "wherefore", "verily", "lo", "behold", "yea", "nay",
    "moreover", "nevertheless", "albeit", "notwithstanding", "whereas", "whereunto"
]
sw.extend(bible_words)

In [ ]:
def clean_text(x):
    x = x.lower()
    words = word_tokenize(x)
    words = [w for w in words if w not in sw]
    words = [w for w in words if w.isalpha()]
    return " ".join(words)

In [ ]:
#df = pd.read_csv('../../../../Data/Text/conference_talks_2000s.txt', sep='\t')
url = 'https://github.com/esnt/Data/raw/main/Text/conference_talks_2000s.txt'
df = pd.read_csv(url, sep='\t')
df.dropna(inplace=True)

In [ ]:
df.tail()

In [ ]:
text = df['talk'].apply(clean_text)

In [ ]:
df.loc[0:5,'talk']

In [ ]:
text[0:5]

In [ ]:
tf = TfidfVectorizer(max_df=.7, min_df=200)
X = tf.fit_transform(text)

In [ ]:
X.shape  

In [ ]:
# Xdf = pd.DataFrame(X.toarray(), columns=tf.get_feature_names_out())
# Xdf.index = df['year']
# Xdf.head(3)
# year_max = Xdf.groupby(level=0).sum()
# year_max.idxmax(axis=1)
# for index, row in year_max.iterrows():
#     print(index)
#     print(row.nlargest(3))
#     print('\n') 

In [ ]:
um = UMAP(2)
vecs = um.fit_transform(X.T)

In [ ]:
plot_df = pd.DataFrame(vecs, columns=['v1','v2'])
plot_df['text'] = tf.get_feature_names_out()

In [ ]:
fig = px.scatter(data_frame=plot_df, x='v1', y='v2', text='text')
fig.update_traces(textposition='top center')
fig.update_layout(width=1500, height=1000)

In [ ]:
## word embeddings.
##  Data should be tokenized by sentence, then by word

In [ ]:
tokenized_sentences = []
for doc in df['talk']:
    sentences = sent_tokenize(doc)
    for sentence in sentences:
        words = simple_preprocess(sentence)
        tokenized_sentences.append(words)

In [ ]:
### Train model with our data

In [ ]:
model = Word2Vec(tokenized_sentences, vector_size=100, window=5, min_count=1)

In [ ]:
model.wv.most_similar('atone', topn=5)

In [ ]:
# word_vectors = model.wv.vectors
# words = list(model.wv.index_to_key)
tf_word_vectors = [model.wv[word] for word in tf.get_feature_names_out() if word in model.wv]
# Reduce dimensions
um2 = UMAP(n_neighbors=5, min_dist=0.3, n_components=2)
umap_embeddings = um2.fit_transform(tf_word_vectors)

In [ ]:
data = {
    "word": tf.get_feature_names_out(),
    "dim1": umap_embeddings[:, 0],
    "dim2": umap_embeddings[:, 1]
}
we_df = pd.DataFrame(data)

# Create the plot
fig = px.scatter(we_df, x="dim1", y="dim2", text="word")
fig.update_traces(textposition='top center')
fig.update_layout(width=1500, height=1000)

# Show plot
fig.show()

In [ ]:
model.wv.most_similar(positive=['woman','king'], negative=['man'], topn=5)

In [ ]:
### Use pretrained model

In [ ]:
## Download from https://github.com/harmanpreet93/load-word2vec-google?tab=readme-ov-file
## (file is 1.6GB)

# from gensim.models import KeyedVectors
# pretrained_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
##--------
## OR
##--------
## code takes about 2 minutes to run

# import gensim.downloader as api
# pretrained_model = api.load('word2vec-google-news-300')

In [ ]:
pretrained_model.most_similar(positive=['woman','king'], negative=['man'], topn=5)

In [ ]:
tf_wv_pretrain = [pretrained_model[word] for word in tf.get_feature_names_out() if word in pretrained_model]
words = [word for word in tf.get_feature_names_out() if word in pretrained_model]
# Reduce dimensions
um2 = UMAP(n_neighbors=5, min_dist=0.3, n_components=2)
umap_embeddings2 = um2.fit_transform(tf_wv_pretrain)

In [ ]:
data = {
    "word": words,
    "dim1": umap_embeddings2[:, 0],
    "dim2": umap_embeddings2[:, 1]
}
we_df2 = pd.DataFrame(data)

# Create the plot
fig = px.scatter(we_df2, x="dim1", y="dim2", text="word")
fig.update_traces(textposition='top center')
fig.update_layout(width=1500, height=1000)

# Show plot
fig.show()